scraping

In [1]:
pip install requests beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


In [2]:
import requests
from bs4 import BeautifulSoup
import time
import random
import urllib.parse # Pour encoder la requête pour l'URL

# --- Configuration ---
# Requête de recherche : cible les articles LinkedIn Pulse sur l'IA
search_query = 'site:linkedin.com/pulse/ "intelligence artificielle" OR "machine learning"'
# Nombre de pages de résultats à explorer (10 résultats par page environ)
num_pages_to_scrape = 3
# User agent pour simuler un navigateur
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36'
}

# --- Stockage des URLs ---
found_linkedin_urls = set() # Utilise un set pour éviter les doublons

# --- Boucle sur les pages de résultats ---
for page in range(num_pages_to_scrape):
    start_index = page * 10
    # Encoder la requête pour l'URL Google
    encoded_query = urllib.parse.quote_plus(search_query)
    google_url = f"https://www.google.com/search?q={encoded_query}&start={start_index}"

    print(f"Scraping Google Search Page {page + 1}: {google_url}")

    try:
        # Faire la requête HTTP
        response = requests.get(google_url, headers=headers)
        response.raise_for_status() # Vérifier si la requête a réussi

        # Analyser le HTML
        soup = BeautifulSoup(response.text, 'html.parser')

        # Trouver les balises contenant les liens de résultats
        # ATTENTION : Les sélecteurs ('div', class_='...) peuvent changer! Inspectez le code source
        # de Google pour trouver les bons sélecteurs si cela ne fonctionne pas.
        link_tags = soup.find_all('a') # Simple approche : prendre tous les liens

        for tag in link_tags:
            href = tag.get('href')

            # Nettoyer et filtrer les URLs Google pour extraire les liens LinkedIn
            if href and href.startswith('/url?q='):
                # Extrait l'URL réelle de la redirection Google
                clean_url = urllib.parse.unquote(href.split('/url?q=')[1].split('&sa=')[0])

                # Vérifier si c'est bien une URL LinkedIn Pulse visée
                if 'linkedin.com/pulse/article/' in clean_url:
                    found_linkedin_urls.add(clean_url)
                    # print(f"  -> Found potential URL: {clean_url}") # Décommentez pour voir chaque URL trouvée

        # Pause polie pour ne pas surcharger Google
        time.sleep(random.uniform(2, 5)) # Attendre entre 2 et 5 secondes

    except requests.exceptions.RequestException as e:
        print(f"  Error during requests to {google_url}: {e}")
        # Possibilité de s'arrêter ou de réessayer ici
        break
    except Exception as e:
        print(f"  An unexpected error occurred: {e}")

# --- Afficher les résultats ---
print(f"\n--- Total Unique LinkedIn Pulse URLs Found: {len(found_linkedin_urls)} ---")
for url in sorted(list(found_linkedin_urls)):
    print(url)

# --- ÉTAPE SUIVANTE ---
# Maintenant, il faut décider comment obtenir le contenu complet de ces URLs.
# Option 1: Visiter manuellement quelques URLs pour évaluer la qualité.
# Option 2: Coder la partie scraping de LinkedIn (avec prudence).
# Option 3: Chercher si ces articles existent sur d'autres plateformes plus faciles à scraper.
print("\n--- Next Step: Develop strategy to fetch full content from these URLs ---")

Scraping Google Search Page 1: https://www.google.com/search?q=site%3Alinkedin.com%2Fpulse%2F+%22intelligence+artificielle%22+OR+%22machine+learning%22&start=0
Scraping Google Search Page 2: https://www.google.com/search?q=site%3Alinkedin.com%2Fpulse%2F+%22intelligence+artificielle%22+OR+%22machine+learning%22&start=10
Scraping Google Search Page 3: https://www.google.com/search?q=site%3Alinkedin.com%2Fpulse%2F+%22intelligence+artificielle%22+OR+%22machine+learning%22&start=20

--- Total Unique LinkedIn Pulse URLs Found: 0 ---

--- Next Step: Develop strategy to fetch full content from these URLs ---


In [3]:
pip install googlesearch-python

Note: you may need to restart the kernel to use updated packages.


In [1]:
from googlesearch import search
import time

# --- Configuration ---
# Requête de recherche : cible les articles LinkedIn Pulse sur l'IA
search_query = 'site:linkedin.com/pulse/ "intelligence artificielle" OR "machine learning"'
# Nombre total de résultats souhaités
num_results = 50 # Essayons d'en obtenir 50 (la bibliothèque gère la pagination)
# Délai entre les requêtes à Google (en secondes) pour être poli
pause_duration = 3.0

# --- Stockage des URLs ---
found_linkedin_urls = set() # Utilise un set pour éviter les doublons

print(f"Recherche Google en cours pour : '{search_query}'")
print(f"Tentative de récupérer jusqu'à {num_results} résultats...")

try:
    # Utilisation de la fonction search de la bibliothèque
    # num : nombre de résultats à récupérer par requête interne
    # stop : nombre total de résultats à récupérer
    # pause : délai entre les requêtes HTTP
    for url in search(search_query, num=10, stop=num_results, pause=pause_duration):
        # Filtrer pour ne garder que les URLs LinkedIn Pulse pertinentes
        # Adaptez si vous ciblez autre chose que /pulse/article/
        if 'linkedin.com/pulse/article/' in url:
            if url not in found_linkedin_urls:
              print(f"  -> Trouvé : {url}")
              found_linkedin_urls.add(url)
        # Petite pause supplémentaire pour être sûr (optionnel)
        # time.sleep(0.5)

except Exception as e:
    # La bibliothèque peut parfois lever des erreurs si Google bloque
    print(f"\nUne erreur est survenue pendant la recherche Google : {e}")
    print("Cela peut être dû à un blocage temporaire de Google.")

# --- Afficher les résultats ---
print(f"\n--- Total Unique LinkedIn Pulse URLs Found: {len(found_linkedin_urls)} ---")
if not found_linkedin_urls:
    print("Aucune URL correspondante trouvée. Essayez d'ajuster la requête de recherche ('search_query').")
else:
    # Optionnel : Sauvegarder les URLs dans un fichier
    try:
        with open("linkedin_pulse_urls.txt", "w") as f:
            for url in sorted(list(found_linkedin_urls)):
                f.write(url + "\n")
        print("Les URLs trouvées ont été sauvegardées dans 'linkedin_pulse_urls.txt'")
    except Exception as e:
        print(f"Erreur lors de la sauvegarde des URLs : {e}")


# --- ÉTAPE SUIVANTE ---
print("\n--- Next Step: Evaluate the found URLs and develop strategy to fetch full content ---")

Recherche Google en cours pour : 'site:linkedin.com/pulse/ "intelligence artificielle" OR "machine learning"'
Tentative de récupérer jusqu'à 50 résultats...

Une erreur est survenue pendant la recherche Google : search() got an unexpected keyword argument 'num'
Cela peut être dû à un blocage temporaire de Google.

--- Total Unique LinkedIn Pulse URLs Found: 0 ---
Aucune URL correspondante trouvée. Essayez d'ajuster la requête de recherche ('search_query').

--- Next Step: Evaluate the found URLs and develop strategy to fetch full content ---


## Plan de Collecte et Préparation des Données pour le Nouveau Fine-Tuning IA

**Objectif :** Créer un jeu de données de haute qualité composé de posts LinkedIn pertinents et engageants sur l'Intelligence Artificielle, afin d'affiner un modèle GPT plus performant et flexible dans ce domaine.

---

### Phase 1 : Découverte des URLs via Google Search

* **But :** Identifier un maximum d'URLs de posts ou d'articles LinkedIn pertinents sur divers sujets liés à l'IA (IA générale, IA générative, Acte IA, actus, etc.).
* **Méthode :** Utilisation d'un script Python avec la bibliothèque `googlesearch-python` pour interroger Google Search avec des mots-clés et des opérateurs `site:linkedin.com`.
* **Entrée :** Requêtes de recherche Google larges couvrant les thèmes IA visés.
* **Sortie :** Un fichier texte (`linkedin_urls_found.txt`) contenant une liste d'URLs LinkedIn potentiellement intéressantes.
* **Code :** Premier script dans ce notebook (`scraping_preparation.ipynb`).

---

### Phase 2 : Scraping du Contenu et de l'Engagement sur LinkedIn

* **But :** Pour chaque URL identifiée en Phase 1, récupérer le texte complet du post/article ainsi que ses métriques d'engagement (nombre de likes, de commentaires).
* **Méthode :** Développer un script Python pour visiter chaque URL LinkedIn. Utilisation de techniques de scraping web (ex: `requests` + `BeautifulSoup4`, ou potentiellement `Selenium`/`Playwright` si nécessaire) pour analyser le HTML de la page LinkedIn et extraire les informations désirées.
* **Entrée :** Le fichier `linkedin_urls_found.txt` de la Phase 1.
* **Sortie :** Un fichier structuré (ex: CSV, JSON) contenant les URLs, le texte des posts, et les nombres de likes/commentaires.
* **⚠️ Attention :** Phase techniquement complexe, potentiellement fragile (changements de structure LinkedIn), et nécessitant de la prudence vis-à-vis des conditions d'utilisation de LinkedIn.

---

### Phase 3 : Filtrage des Données et Formatage pour OpenAI

* **But :** Sélectionner les posts les plus "pertinents" (basé sur l'engagement) et les formater correctement pour l'API de fine-tuning d'OpenAI.
* **Méthode :**
    1.  Analyser les données de la Phase 2 pour définir un seuil d'engagement (ex: garder les posts avec > N likes/commentaires).
    2.  Filtrer le jeu de données pour ne conserver que ces posts "performants".
    3.  Convertir le contenu textuel sélectionné au format JSONL attendu par OpenAI (généralement une structure de messages `{"messages": [{"role": "system", ...}, {"role": "user", ...}, {"role": "assistant", "content": "TEXTE_DU_POST"}]}`).
* **Entrée :** Les données structurées de la Phase 2.
* **Sortie :** Le fichier final `fine-tuning-data-ai.jsonl` (ou nom similaire) prêt à être uploadé sur OpenAI pour le nouveau job de fine-tuning.

---

In [2]:
# Étape 1 : Installer la bibliothèque si ce n'est pas fait
# !pip install googlesearch-python

from googlesearch import search
import time

# --- Configuration ---
# Requête de recherche élargie pour divers sujets IA sur LinkedIn
search_query = (
    'site:linkedin.com/pulse/ ("intelligence artificielle" OR "IA générative" OR "generative AI" OR "EU AI Act" OR "Acte IA Europe" OR "actualité IA" OR "machine learning") '
    'OR site:linkedin.com/posts/ ("intelligence artificielle" OR "IA générative" OR "generative AI" OR "EU AI Act" OR "Acte IA Europe" OR "actualité IA" OR "machine learning")'
)
# Nombre total de résultats souhaités (ajustez si nécessaire)
num_results = 100 # Essayons d'en obtenir 100
# Délai entre les requêtes à Google (en secondes) - Important !
pause_duration = 4.0 # Augmenté à 4 secondes pour être plus prudent

# --- Stockage des URLs ---
found_linkedin_urls = set()

print(f"Recherche Google élargie en cours pour : '{search_query}'")
print(f"Tentative de récupérer jusqu'à {num_results} résultats...")

try:
    # Utilisation de la fonction search corrigée (num_results au lieu de num)
    for url in search(search_query, num_results=10, stop=num_results, pause=pause_duration):
        # Filtrer pour ne garder que les URLs LinkedIn (Pulse ou Posts)
        if 'linkedin.com/' in url:
            if '/groups/' not in url and '/jobs/' not in url and '/company/' not in url: # Exclure certains types
                if url not in found_linkedin_urls:
                  print(f"  -> Trouvé : {url}")
                  found_linkedin_urls.add(url)
        # time.sleep(0.2) # Petite pause supplémentaire (optionnel)

except Exception as e:
    print(f"\nUne erreur est survenue pendant la recherche Google : {e}")
    print("Cela peut être dû à un blocage temporaire de Google ou à un autre problème.")

# --- Afficher les résultats ---
print(f"\n--- Total Unique LinkedIn URLs Found: {len(found_linkedin_urls)} ---")
if not found_linkedin_urls:
    print("Aucune URL correspondante trouvée. Essayez d'ajuster ou de simplifier la requête de recherche ('search_query'), ou augmentez 'num_results'.")
else:
    # Sauvegarder les URLs dans un fichier
    output_filename = "linkedin_urls_found.txt" # Sauvegardé dans le dossier 01_collecte_donnees
    try:
        with open(output_filename, "w") as f:
            for url in sorted(list(found_linkedin_urls)):
                f.write(url + "\n")
        print(f"Les URLs trouvées ont été sauvegardées dans '{output_filename}'")
    except Exception as e:
        print(f"Erreur lors de la sauvegarde des URLs : {e}")


# --- Phase 2 (Code Suivant) ---
print(f"\n--- Prochaine Étape (Phase 2) : Développer le code pour visiter les {len(found_linkedin_urls)} URLs trouvées, ---")
print("--- extraire le contenu complet des posts ET les données d'engagement (likes/commentaires). ---")
print("--- Attention : Cette étape nécessitera du scraping direct de LinkedIn et sera plus complexe. ---")

Recherche Google élargie en cours pour : 'site:linkedin.com/pulse/ ("intelligence artificielle" OR "IA générative" OR "generative AI" OR "EU AI Act" OR "Acte IA Europe" OR "actualité IA" OR "machine learning") OR site:linkedin.com/posts/ ("intelligence artificielle" OR "IA générative" OR "generative AI" OR "EU AI Act" OR "Acte IA Europe" OR "actualité IA" OR "machine learning")'
Tentative de récupérer jusqu'à 100 résultats...

Une erreur est survenue pendant la recherche Google : search() got an unexpected keyword argument 'stop'
Cela peut être dû à un blocage temporaire de Google ou à un autre problème.

--- Total Unique LinkedIn URLs Found: 0 ---
Aucune URL correspondante trouvée. Essayez d'ajuster ou de simplifier la requête de recherche ('search_query'), ou augmentez 'num_results'.

--- Prochaine Étape (Phase 2) : Développer le code pour visiter les 0 URLs trouvées, ---
--- extraire le contenu complet des posts ET les données d'engagement (likes/commentaires). ---
--- Attention : C

In [3]:
# Étape 1 : Installer la bibliothèque si ce n'est pas fait
# !pip install googlesearch-python

from googlesearch import search
import time

# --- Configuration ---
search_query = (
    'site:linkedin.com/pulse/ ("intelligence artificielle" OR "IA générative" OR "generative AI" OR "EU AI Act" OR "Acte IA Europe" OR "actualité IA" OR "machine learning") '
    'OR site:linkedin.com/posts/ ("intelligence artificielle" OR "IA générative" OR "generative AI" OR "EU AI Act" OR "Acte IA Europe" OR "actualité IA" OR "machine learning")'
)
# Limite manuelle du nombre de résultats Google à traiter
results_limit = 100
# Délai entre les requêtes à Google (en secondes) - Important !
pause_duration = 4.0

# --- Stockage des URLs ---
found_linkedin_urls = set()
processed_google_results = 0 # Compteur manuel

print(f"Recherche Google élargie en cours pour : '{search_query}'")
print(f"Traitement d'au maximum {results_limit} résultats Google...")

try:
    # Appel simplifié de la fonction search (uniquement query et pause)
    for url in search(search_query, pause=pause_duration):
        processed_google_results += 1
        # Filtrer pour ne garder que les URLs LinkedIn pertinentes
        if 'linkedin.com/' in url:
            if '/groups/' not in url and '/jobs/' not in url and '/company/' not in url: # Exclure certains types
                if url not in found_linkedin_urls:
                  print(f"  -> Trouvé ({processed_google_results}): {url}")
                  found_linkedin_urls.add(url)

        # Arrêter manuellement si on a traité assez de résultats Google
        if processed_google_results >= results_limit:
            print(f"\nLimite de {results_limit} résultats Google traités atteinte.")
            break

except Exception as e:
    print(f"\nUne erreur est survenue pendant la recherche Google : {e}")
    print("Cela peut être dû à un blocage temporaire de Google ou à un autre problème.")

# --- Afficher les résultats ---
print(f"\n--- Total Unique LinkedIn URLs Found: {len(found_linkedin_urls)} ---")
if not found_linkedin_urls:
    print(f"Aucune URL LinkedIn pertinente trouvée parmi les {processed_google_results} premiers résultats Google.")
    print("Essayez d'ajuster ou de simplifier la requête de recherche ('search_query').")
else:
    # Sauvegarder les URLs dans un fichier
    output_filename = "linkedin_urls_found.txt"
    try:
        with open(output_filename, "w") as f:
            for url in sorted(list(found_linkedin_urls)):
                f.write(url + "\n")
        print(f"Les URLs trouvées ont été sauvegardées dans '{output_filename}'")
    except Exception as e:
        print(f"Erreur lors de la sauvegarde des URLs : {e}")


# --- Phase 2 (Code Suivant) ---
print(f"\n--- Prochaine Étape (Phase 2) : Développer le code pour visiter les {len(found_linkedin_urls)} URLs trouvées, ---")
print("--- extraire le contenu complet des posts ET les données d'engagement (likes/commentaires). ---")
print("--- Attention : Cette étape nécessitera du scraping direct de LinkedIn et sera plus complexe. ---")

Recherche Google élargie en cours pour : 'site:linkedin.com/pulse/ ("intelligence artificielle" OR "IA générative" OR "generative AI" OR "EU AI Act" OR "Acte IA Europe" OR "actualité IA" OR "machine learning") OR site:linkedin.com/posts/ ("intelligence artificielle" OR "IA générative" OR "generative AI" OR "EU AI Act" OR "Acte IA Europe" OR "actualité IA" OR "machine learning")'
Traitement d'au maximum 100 résultats Google...

Une erreur est survenue pendant la recherche Google : search() got an unexpected keyword argument 'pause'
Cela peut être dû à un blocage temporaire de Google ou à un autre problème.

--- Total Unique LinkedIn URLs Found: 0 ---
Aucune URL LinkedIn pertinente trouvée parmi les 0 premiers résultats Google.
Essayez d'ajuster ou de simplifier la requête de recherche ('search_query').

--- Prochaine Étape (Phase 2) : Développer le code pour visiter les 0 URLs trouvées, ---
--- extraire le contenu complet des posts ET les données d'engagement (likes/commentaires). ---
-

In [4]:
# Étape 1 : Installer la bibliothèque si ce n'est pas fait
# !pip install googlesearch-python

from googlesearch import search
import time

# --- Configuration ---
search_query = (
    'site:linkedin.com/pulse/ ("intelligence artificielle" OR "IA générative" OR "generative AI" OR "EU AI Act" OR "Acte IA Europe" OR "actualité IA" OR "machine learning") '
    'OR site:linkedin.com/posts/ ("intelligence artificielle" OR "IA générative" OR "generative AI" OR "EU AI Act" OR "Acte IA Europe" OR "actualité IA" OR "machine learning")'
)
# On ne contrôle plus la limite via les arguments
print(f"Recherche Google élargie en cours pour : '{search_query}'")
print("Tentative avec appel EXTRÊMEMENT simplifié (sans pause/stop/num)...")

# --- Stockage des URLs ---
found_linkedin_urls = set()
processed_google_results = 0

try:
    # APPEL EXTRÊMEMENT SIMPLIFIÉ - Uniquement l'argument query
    for url in search(search_query):
        processed_google_results += 1
        # Filtrer pour ne garder que les URLs LinkedIn pertinentes
        if 'linkedin.com/' in url:
            if '/groups/' not in url and '/jobs/' not in url and '/company/' not in url:
                if url not in found_linkedin_urls:
                  print(f"  -> Trouvé ({processed_google_results}): {url}")
                  found_linkedin_urls.add(url)
        # On ne peut pas contrôler la pause entre les requêtes Google internes ici
        # Une pause ici ne fait que ralentir le traitement des résultats déjà reçus
        time.sleep(0.1)

    # NOTE: On ne sait pas combien de résultats Google la bibliothèque a réellement tenté de récupérer

except Exception as e:
    print(f"\nUne erreur est survenue pendant la recherche Google : {e}")
    print("Cela peut être dû à un blocage temporaire de Google ou à un problème avec la bibliothèque.")
    # Regardez si le message d'erreur donne des indices sur les arguments acceptés

# --- Afficher les résultats ---
print(f"\n--- Total Unique LinkedIn URLs Found: {len(found_linkedin_urls)} ({processed_google_results} résultats Google traités) ---")
if not found_linkedin_urls:
    print(f"Aucune URL LinkedIn pertinente trouvée.")
    print("La bibliothèque ne retourne peut-être pas de résultats ou le filtrage est trop strict.")
else:
    # Sauvegarder les URLs dans un fichier
    output_filename = "linkedin_urls_found.txt"
    try:
        with open(output_filename, "w") as f:
            for url in sorted(list(found_linkedin_urls)):
                f.write(url + "\n")
        print(f"Les URLs trouvées ont été sauvegardées dans '{output_filename}'")
    except Exception as e:
        print(f"Erreur lors de la sauvegarde des URLs : {e}")


# --- Phase 2 (Code Suivant) ---
print(f"\n--- Prochaine Étape (Phase 2) : Développer le code pour visiter les {len(found_linkedin_urls)} URLs trouvées, ---")
print("--- extraire le contenu complet des posts ET les données d'engagement (likes/commentaires). ---")
print("--- Attention : Cette étape nécessitera du scraping direct de LinkedIn et sera plus complexe. ---")

Recherche Google élargie en cours pour : 'site:linkedin.com/pulse/ ("intelligence artificielle" OR "IA générative" OR "generative AI" OR "EU AI Act" OR "Acte IA Europe" OR "actualité IA" OR "machine learning") OR site:linkedin.com/posts/ ("intelligence artificielle" OR "IA générative" OR "generative AI" OR "EU AI Act" OR "Acte IA Europe" OR "actualité IA" OR "machine learning")'
Tentative avec appel EXTRÊMEMENT simplifié (sans pause/stop/num)...
  -> Trouvé (1): https://fr.linkedin.com/pulse/intelligence-artificielle-g%C3%A9n%C3%A9rative-r%C3%A9volution-et-applications-urdhe
  -> Trouvé (2): https://fr.linkedin.com/pulse/machine-learning-llm-ia-g%C3%A9n%C3%A9rative-ce-que-ces-types-dia-ribeiro-aiyff
  -> Trouvé (3): https://fr.linkedin.com/pulse/lia-g%C3%A9n%C3%A9rative-r%C3%A9volution-ou-simple-%C3%A9volution-maltem-africa-2ahae
  -> Trouvé (4): https://fr.linkedin.com/pulse/lintelligence-artificielle-g%C3%A9n%C3%A9rative-un-nouveau-n8q6e
  -> Trouvé (5): https://fr.linkedin.com/pulse

### Approche pour le Développement de la Phase 2 : Scraping LinkedIn

**Contexte :** La Phase 1 nous a permis de collecter une première liste d'URLs LinkedIn potentielles via Google Search. La Phase 2 vise à visiter ces URLs pour extraire le contenu texte complet et les métriques d'engagement (likes, commentaires). Cette étape de scraping direct de LinkedIn est connue pour être complexe et sensible aux changements de structure du site.

**Stratégie Adoptée : Développement Incrémental**

1.  **Tester sur un Petit Échantillon :** Nous allons développer et déboguer le code Python de la Phase 2 en utilisant **uniquement le petit nombre d'URLs (~10) collectées lors du premier test de la Phase 1**.
2.  **Valider la Faisabilité :** L'objectif est de prouver qu'il est possible d'extraire les informations souhaitées (texte, likes, commentaires) pour cet échantillon réduit.
3.  **Itérer Rapidement :** Travailler avec peu d'URLs permet de tester les changements de code rapidement et de comprendre les difficultés spécifiques du scraping de LinkedIn sans lancer de nombreuses requêtes potentiellement bloquantes.

**Avantages de cette Approche :**

* Réduit le temps de débogage.
* Minimise le risque de blocage par LinkedIn pendant la phase de développement.
* Permet de valider le concept avant d'investir du temps dans la collecte d'une grande quantité d'URLs (Phase 1 à grande échelle).

**Plan Post-Validation :**

* Une fois que le script de la Phase 2 fonctionnera de manière fiable sur le petit échantillon :
    * **Améliorer la Phase 1 :** Optimiser ou relancer la collecte d'URLs pour obtenir une liste beaucoup plus conséquente (plusieurs centaines).
    * **Exécuter Phase 2 à l'Échelle :** Lancer le script de scraping validé sur la grande liste d'URLs (avec les précautions nécessaires : pauses, gestion des erreurs, etc.).
    * **Procéder à la Phase 3 :** Filtrer et formater les données collectées en masse.

**Action Immédiate :** Nous commençons maintenant le développement du code de la Phase 2 en ciblant les URLs présentes dans `linkedin_urls_found.txt`.

In [9]:
import requests
from bs4 import BeautifulSoup # On l'importe déjà pour la suite
import time
import random
import os # Pour vérifier l'existence du fichier

# --- Configuration ---
urls_filename = "linkedin_urls_found.txt" # Fichier créé par le script de Phase 1
target_url = None # Contiendra l'URL à tester

# --- 1. Charger les URLs depuis le fichier ---
linkedin_urls = []
# Vérifier si le fichier existe dans le dossier courant (01_collecte_donnees)
if os.path.exists(urls_filename):
    try:
        with open(urls_filename, "r") as f:
            # Lire chaque ligne, enlever les espaces/sauts de ligne, ignorer lignes vides
            linkedin_urls = [line.strip() for line in f if line.strip()]
        if len(linkedin_urls) >= 2: # Vérifier qu'il y a au moins 2 URLs
            print(f"✅ {len(linkedin_urls)} URLs chargées depuis '{urls_filename}'")
            # ---- MODIFICATION IMPORTANTE ICI ----
            target_url = linkedin_urls[1] # Sélectionner la DEUXIÈME URL pour ce test
            # ------------------------------------
            print(f" ciblant l'URL : {target_url}")
        elif linkedin_urls:
             print(f"⚠️ Fichier '{urls_filename}' contient seulement {len(linkedin_urls)} URL(s). Impossible de tester la deuxième.")
        else:
            print(f"⚠️ Le fichier '{urls_filename}' est vide. Impossible de continuer.")
    except Exception as e:
        print(f"❌ Erreur lors de la lecture du fichier '{urls_filename}': {e}")
else:
    print(f"❌ Erreur : Le fichier '{urls_filename}' n'a pas été trouvé.")
    print("   Assurez-vous que le script de la Phase 1 a bien fonctionné et créé le fichier.")

# --- 2. Essayer de télécharger la page de l'URL cible (si une URL a été chargée) ---
if target_url:
    print("\nTentative de téléchargement de la page LinkedIn...")
    # Simuler un navigateur commun pour éviter blocage immédiat
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36',
        'Accept-Language': 'fr-FR,fr;q=0.9,en-US;q=0.8,en;q=0.7' # Préférer le français
    }

    try:
        # Envoyer la requête GET avec un timeout de 15 secondes
        response = requests.get(target_url, headers=headers, timeout=15)

        # Vérifier si la requête a échoué (code 4xx ou 5xx)
        response.raise_for_status()

        print(f"✅ Succès ! Code de statut : {response.status_code}")
        print(f"\n--- Extrait du début du code HTML reçu (500 caractères) ---")
        # Afficher le début du HTML pour voir si ça ressemble à une page LinkedIn ou une page d'erreur/login
        print(response.text[:500])
        print("--- Fin de l'extrait ---")

        # --- Prochaine étape : Analyser le HTML avec BeautifulSoup ---
        print("\n➡️ Prochaine étape : Analyser ce code HTML avec BeautifulSoup pour trouver le contenu.")

        # Analyser le HTML avec BeautifulSoup (Partie à activer et adapter après inspection)
        try:
            soup = BeautifulSoup(response.text, 'html.parser')
            print("\n✅ HTML analysé avec BeautifulSoup (initialisation).")

            # --- Extraction du Contenu de l'Article ---
            # !! REMPLACEZ 'div' et 'class_' par ce que vous avez trouvé lors de l'inspection !!
            article_body_tag = soup.find('div', class_='classe-du-corps-article-a-trouver') # EXEMPLE, À CHANGER
            if article_body_tag:
                article_text = article_body_tag.get_text(separator='\n', strip=True)
                print("\n--- Texte de l'Article (Début) ---")
                print(article_text[:1000]) # Afficher les 1000 premiers caractères
                print("--- Fin de l'extrait ---")
            else:
                print("⚠️ Corps de l'article non trouvé avec le sélecteur actuel.")
                print("   Veuillez inspecter la page CIBLE dans votre navigateur et adapter le sélecteur ('find').")

            # --- Extraction des Likes/Réactions ---
            # !! REMPLACEZ le sélecteur par ce que vous avez trouvé !!
            likes_tag = soup.find('span', class_='classe-pour-les-likes-a-trouver') # EXEMPLE, À CHANGER
            if likes_tag:
                likes_text = likes_tag.get_text(strip=True).split()[0]
                print(f"\n--- Likes/Réactions trouvés : {likes_text}")
            else:
                print("⚠️ Likes/Réactions non trouvés avec le sélecteur actuel.")
                print("   Veuillez inspecter la page CIBLE et adapter le sélecteur.")


            # --- Extraction du Nombre de Commentaires ---
            # !! REMPLACEZ le sélecteur par ce que vous avez trouvé !!
            comments_tag = soup.find('a', class_='classe-pour-commentaires-a-trouver') # EXEMPLE, À CHANGER
            if comments_tag:
                comments_text = comments_tag.get_text(strip=True).split()[0]
                print(f"\n--- Commentaires trouvés : {comments_text}")
            else:
                print("⚠️ Commentaires non trouvés avec le sélecteur actuel.")
                print("   Veuillez inspecter la page CIBLE et adapter le sélecteur.")

        except Exception as e:
            print(f"❌ Erreur pendant l'analyse BeautifulSoup : {e}")
        # --- Fin de l'analyse BeautifulSoup ---

    except requests.exceptions.HTTPError as http_err:
        print(f"❌ Erreur HTTP : {http_err} - Code de statut : {response.status_code}")
        print("   LinkedIn a probablement bloqué la requête ou la page nécessite une connexion.")
        print(f"\n--- Extrait de la page d'erreur reçue (500 caractères) ---")
        print(response.text[:500])
        print("--- Fin de l'extrait ---")
    except requests.exceptions.ConnectionError as conn_err:
        print(f"❌ Erreur de Connexion : {conn_err}")
    except requests.exceptions.Timeout as timeout_err:
        print(f"❌ Délai d'attente dépassé : {timeout_err}")
    except requests.exceptions.RequestException as req_err:
        print(f"❌ Erreur pendant la requête : {req_err}")
    except Exception as e:
         print(f"❌ Une erreur inattendue est survenue : {e}")

else:
    print("\nAucune URL cible sélectionnée ou fichier d'URLs invalide, arrêt.")

✅ 10 URLs chargées depuis 'linkedin_urls_found.txt'
 ciblant l'URL : https://fr.linkedin.com/pulse/derni%C3%A8res-avanc%C3%A9es-en-ia-g%C3%A9n%C3%A9rative-mars-2025-arnault-chatel-zs39e

Tentative de téléchargement de la page LinkedIn...
✅ Succès ! Code de statut : 200

--- Extrait du début du code HTML reçu (500 caractères) ---
<!DOCTYPE html>

    
    
    
    

    

    
    
    
    
    
    <html class="cls-fix-enabled" lang="fr">
      <head>
        <meta name="pageKey" content="d_flagship2_pulse_read">
          
    <meta name="robots" content="max-image-preview:large, noarchive">
    <meta name="bingbot" content="nocache">
  
<!----><!---->        <meta name="locale" content="fr_FR">
        <meta id="config" data-app-version="0.0.4381" data-call-tree-id="AAYzcHtwPgrBQqXYtCd8cQ==" data-multiproduct-name="
--- Fin de l'extrait ---

➡️ Prochaine étape : Analyser ce code HTML avec BeautifulSoup pour trouver le contenu.

✅ HTML analysé avec BeautifulSoup (initialisation).
⚠️

### Phase 2 : Validation du Téléchargement HTML (Test sur URL spécifique)

**Constat Initial :**
La première URL (`index 0`) extraite par le script de la Phase 1 s'est révélée invalide lors d'une visite manuelle dans le navigateur (page "Article non trouvé" puis redirection). Le script Python obtenait bien un code de statut 200, mais pour cette page d'erreur, et non pour l'article réel.

**Action Corrective :**
Pour vérifier si le téléchargement HTML via `requests` était possible sur une URL valide, nous avons :
1.  Vérifié manuellement la **deuxième URL** (`index 1`) du fichier `linkedin_urls_found.txt`. Cette URL menait bien à un article existant ("Les dernières avancées en IA générative...").
2.  Modifié le script Python pour cibler spécifiquement cette **deuxième URL** (`target_url = linkedin_urls[1]`).
3.  Ré-exécuté le script.

**Résultat Obtenu :**
L'exécution du script sur la deuxième URL a réussi :
* Code de statut : **200 OK**.
* L'extrait HTML retourné correspondait bien au début du code source de la page de l'article valide.

**Conclusion de l'Étape :**
Le téléchargement du code HTML brut via la bibliothèque `requests` fonctionne pour les URLs d'articles LinkedIn Pulse valides (au moins pour l'URL testée). Nous pouvons donc maintenant passer à l'étape suivante : utiliser `BeautifulSoup` pour analyser ce HTML et tenter d'en extraire le contenu texte, les likes et les commentaires. L'inspection manuelle de cette URL valide est nécessaire pour trouver les bons sélecteurs HTML.

apres divers soucis niveau scraping linkedin nous testons une nouvelle methode


In [12]:
# --- Imports Nécessaires ---
import time
import random
import sys
import re
import json
import os
from dotenv import load_dotenv # Pour charger le .env
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup

# --- Charger les Variables d'Environnement (depuis le fichier .env) ---
# Assurez-vous que le fichier .env est dans le dossier 01_collecte_donnees
load_dotenv()
LINKEDIN_USERNAME = os.getenv("LINKEDIN_USERNAME")
LINKEDIN_PASSWORD = os.getenv("LINKEDIN_PASSWORD")

# Vérifier si les identifiants ont été chargés
if not LINKEDIN_USERNAME or not LINKEDIN_PASSWORD:
    print("⚠️ERREUR: Identifiants LinkedIn non trouvés dans le fichier .env ou les variables d'environnement.")
    raise ValueError("Identifiants LinkedIn manquants.")
else:
    print(f"✅ Identifiants LinkedIn chargés depuis .env (Utilisateur: {LINKEDIN_USERNAME})")

print("✅ Imports et configuration initiale terminés.")

✅ Identifiants LinkedIn chargés depuis .env (Utilisateur: gonzalez.nicolas@icloud.com)
✅ Imports et configuration initiale terminés.


In [13]:
# --- Fonctions Selenium et Extraction (Mises à jour) ---

def get_user_agent():
    """Retourne un User-Agent aléatoire"""
    user_agents = [
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36',
        'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.0 Safari/605.1.15',
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:123.0) Gecko/20100101 Firefox/123.0',
    ]
    return random.choice(user_agents)

def setup_driver():
    """Configure et retourne un driver Selenium Chrome"""
    print("\nConfiguration du navigateur Chrome (Selenium)...")
    chrome_options = Options()
    # Pour voir le navigateur, commentez la ligne suivante :
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument(f"user-agent={get_user_agent()}")
    chrome_options.add_argument("--disable-notifications")
    chrome_options.add_argument("--disable-extensions")
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--window-size=1920,1080")
    chrome_options.add_argument("--log-level=3")

    driver = None
    try:
        service = Service(ChromeDriverManager().install())
        driver = webdriver.Chrome(service=service, options=chrome_options)
        print("✅ Navigateur Chrome configuré.")
        return driver
    except ValueError as ve:
         print(f"❌ Erreur de configuration WebDriver: {ve}")
         return None
    except Exception as e:
        print(f"❌ Erreur inattendue config navigateur: {e}")
        if driver: driver.quit()
        return None

def linkedin_login(driver, username, password):
    """Se connecte à LinkedIn"""
    if not driver: return False
    print("\nTentative de connexion à LinkedIn...")
    try:
        login_url = "https://www.linkedin.com/login?fromSignIn=true&trk=guest_homepage-basic_nav-header-signin"
        driver.get(login_url)
        print(f" Accès à {login_url}")
        time.sleep(random.uniform(3, 5))

        if "feed" in driver.current_url:
            print(" Déjà connecté.")
            return True

        print(" Remplissage du formulaire...")
        try:
            username_field = WebDriverWait(driver, 15).until(EC.visibility_of_element_located((By.ID, "username")))
            password_field = driver.find_element(By.ID, "password")
            username_field.send_keys(username)
            time.sleep(random.uniform(0.5, 1.2))
            password_field.send_keys(password)
            time.sleep(random.uniform(0.5, 1.2))
            submit_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[data-litms-control-urn='login-submit'], button[type='submit']")))
            submit_button.click()
            print(" Bouton de connexion cliqué.")
            print(" Attente après soumission (5-8 sec)...")
            time.sleep(random.uniform(5, 8))
            current_url = driver.current_url
            if "feed" in current_url:
                print("✅ Connexion réussie.")
                return True
            elif "checkpoint" in current_url:
                print("⚠️ Connexion réussie mais checkpoint détecté.")
                return True # On essaie quand même
            else:
                print(f"❌ Échec de la connexion. URL: {current_url}")
                try:
                    error_element = driver.find_element(By.CSS_SELECTOR, "[role='alert'], .form__input--error, #error-for-password, #error-for-username")
                    if error_element: print(f" Message d'erreur: {error_element.text[:150]}")
                except NoSuchElementException: print(" Aucun message d'erreur trouvé.")
                return False
        except TimeoutException: print("❌ Timeout formulaire connexion."); return False
        except Exception as e: print(f"❌ Erreur formulaire : {e}"); return False
    except Exception as e: print(f"❌ Erreur majeure connexion : {e}"); return False

def get_page_content_with_selenium(url, driver):
    """Récupère le HTML de la page cible avec Selenium"""
    if not driver: return None
    print(f"\nAccès à l'URL cible via Selenium : {url}")
    try:
        driver.get(url)
        print(" Attente chargement (5-8 sec)...")
        time.sleep(random.uniform(5, 8))
        current_url = driver.current_url
        if "login" in current_url or "signup" in current_url or "authwall" in current_url:
             print(f"⚠️ Redirection vers {current_url}.")
             return None
        print(" Défilement léger...")
        for i in range(2):
             driver.execute_script(f"window.scrollTo(0, {(i + 1) * 600});")
             print(f" Scroll {i+1}/2...")
             time.sleep(random.uniform(1.5, 2.5))
        try:
             WebDriverWait(driver, 15).until(EC.visibility_of_element_located((By.TAG_NAME, "h1")))
             print(" Élément H1 visible.")
        except TimeoutException: print("⚠️ Timeout H1.")
        html_content = driver.page_source
        print("✅ Contenu HTML récupéré.")
        return html_content
    except Exception as e: print(f"❌ Erreur récupération page : {e}"); return None

# --- NOUVELLE VERSION ICI ---
def extract_article_content(html_content):
    """Extrait le contenu principal de l'article en utilisant le sélecteur .reader-article-content"""
    if not html_content: return "Contenu HTML non disponible"
    print("\nAnalyse HTML pour contenu article (avec sélecteur .reader-article-content)...")
    soup = BeautifulSoup(html_content, 'html.parser')
    article_text = ""
    found_by = "Aucune méthode"
    selector = 'div.reader-article-content' # <--- NOUVEAU SÉLECTEUR BASÉ SUR INSPECTION

    try:
        content_div = soup.select_one(selector)
        if content_div:
            content_parts = []
            # Itérer sur les éléments pour extraire le texte
            for element in content_div.find_all(['p', 'h1', 'h2', 'h3', 'h4', 'h5', 'h6', 'li'], recursive=True):
                 text_part = element.get_text(strip=True)
                 if text_part: content_parts.append(text_part)
            if content_parts:
                article_text = "\n\n".join(content_parts)
                found_by = f"Sélecteur '{selector}'"
                print(f" Contenu trouvé via: {found_by}")
                return article_text.strip()
            else: # Fallback texte brut
                 article_text = content_div.get_text(separator='\n', strip=True)
                 if article_text:
                      found_by = f"Sélecteur '{selector}' (texte brut)"
                      print(f" Contenu trouvé via: {found_by}")
                      return article_text.strip()
    except Exception as e: print(f" Erreur en cherchant contenu avec '{selector}': {e}"); pass

    if not article_text:
        print("⚠️ Contenu article non trouvé avec nouveau sélecteur.")
        return "Contenu non trouvé"
    return "Contenu non trouvé (fin)" # Ne devrait pas arriver
# --- FIN NOUVELLE VERSION ---

def extract_likes_count_with_selenium(driver):
    """Extrait le nombre de likes/réactions avec Selenium (méthode aria-label)"""
    print("\nRecherche des Likes/Réactions...")
    try:
        xpath_selector = "//button[contains(@aria-label, 'réaction') or contains(@aria-label, 'reaction')]"
        likes_element = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, xpath_selector)))
        aria_label_text = likes_element.get_attribute("aria-label")
        match = re.search(r'^(\d+)', aria_label_text)
        if match:
            likes_count = match.group(1)
            print(f" Likes trouvés via XPath aria-label: '{xpath_selector}' -> {likes_count}")
            return likes_count
        else:
             likes_text = likes_element.get_attribute("textContent").strip()
             match_text = re.search(r'^(\d+)', likes_text)
             if match_text:
                  likes_count = match_text.group(1); print(f" Likes trouvés via XPath (texte fallback): '{xpath_selector}' -> {likes_count}"); return likes_count
    except (NoSuchElementException, TimeoutException): print("⚠️ Élément Likes non trouvé via XPath.")
    except Exception as e: print(f" Erreur mineure likes XPath: {e}")
    # Fallback CSS (au cas où)
    try:
        selector_css = ".social-details-social-counts__reactions-count"; likes_element_css = WebDriverWait(driver, 5).until(EC.visibility_of_element_located((By.CSS_SELECTOR, selector_css))); likes_text_css = likes_element_css.get_attribute("textContent").strip(); match_css = re.search(r'^(\d+)', likes_text_css)
        if match_css: likes_count_css = match_css.group(1); print(f" Likes trouvés via CSS fallback: '{selector_css}' -> {likes_count_css}"); return likes_count_css
    except Exception: print("⚠️ Likes non trouvés via CSS non plus.")
    return "Non trouvé"

def extract_comments_count_with_selenium(driver):
    """Extrait le nombre de commentaires avec Selenium (méthode aria-label)"""
    print("\nRecherche des Commentaires...")
    try:
        xpath_selector = "//button[contains(@aria-label, 'commentaire') or contains(@aria-label, 'comment')]"
        # Alternative : xpath_selector = "//a[contains(@aria-label, 'commentaire') or contains(@aria-label, 'comment')]"
        comments_element = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, xpath_selector)))
        aria_label_text = comments_element.get_attribute("aria-label")
        match = re.search(r'^(\d+)', aria_label_text)
        if match:
            comments_count = match.group(1); print(f" Commentaires trouvés via XPath aria-label: '{xpath_selector}' -> {comments_count}"); return comments_count
        else:
             comments_text = comments_element.get_attribute("textContent").strip(); match_text = re.search(r'^(\d+)', comments_text)
             if match_text: comments_count = match_text.group(1); print(f" Commentaires trouvés via XPath (texte fallback): '{xpath_selector}' -> {comments_count}"); return comments_count
    except (NoSuchElementException, TimeoutException): print("⚠️ Élément Commentaires non trouvé via XPath.")
    except Exception as e: print(f" Erreur mineure commentaires XPath: {e}")
    # Ajouter fallbacks CSS si nécessaire
    print("⚠️ Nombre de commentaires non trouvé via sélecteurs principaux.")
    return "Non trouvé"

print("✅ Fonctions Selenium (avec extract_article_content mise à jour) définies.")

✅ Fonctions Selenium (avec extract_article_content mise à jour) définies.


### Phase 2 : Succès de l'Extraction Initiale !

**Bilan de l'Exécution Précédente :**
Le script Selenium, exécuté sur la **deuxième URL** de test (article sur l'IA générative), a réussi à :
1.  Se connecter à LinkedIn avec les identifiants du fichier `.env`.
2.  Accéder à la page de l'article après la connexion.
3.  Récupérer le code HTML complet de la page.
4.  Utiliser les fonctions d'extraction (avec les sélecteurs affinés) pour obtenir :
    * ✅ Le **contenu** principal de l'article (via `div.reader-article-content`).
    * ✅ Le nombre de **likes/réactions** (36, via XPath/aria-label).
    * ✅ Le nombre de **commentaires** (16, via XPath/aria-label).

**Conclusion :** Le processus d'extraction pour une URL unique est maintenant fonctionnel !

---

**Prochaine Étape : Structurer les Données Extraites**

Avant de modifier le script pour traiter *toutes* les URLs de la liste, nous allons d'abord améliorer la façon dont les résultats sont gérés pour une seule URL.

**Action immédiate :** Modifier la fin du code de la cellule d'exécution précédente pour que les informations extraites (URL, contenu, likes, commentaires) soient stockées dans un **dictionnaire Python** au lieu d'être simplement affichées séparément. Cela facilitera ensuite la collecte des résultats pour plusieurs URLs.

In [28]:
# --- Imports Nécessaires ---
import time
import random
import sys
import re
import json
import os
from dotenv import load_dotenv # Pour charger le .env
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup

# --- Charger les Variables d'Environnement (depuis le fichier .env) ---
# Assurez-vous que le fichier .env est dans le dossier 01_collecte_donnees
load_dotenv()
LINKEDIN_USERNAME = os.getenv("LINKEDIN_USERNAME")
LINKEDIN_PASSWORD = os.getenv("LINKEDIN_PASSWORD")

# Vérifier si les identifiants ont été chargés
if not LINKEDIN_USERNAME or not LINKEDIN_PASSWORD:
    print("⚠️ERREUR: Identifiants LinkedIn non trouvés dans le fichier .env ou les variables d'environnement.")
    raise ValueError("Identifiants LinkedIn manquants.")
else:
    print(f"✅ Identifiants LinkedIn chargés depuis .env (Utilisateur: {LINKEDIN_USERNAME})")

print("✅ Imports et configuration initiale terminés.")

✅ Identifiants LinkedIn chargés depuis .env (Utilisateur: gonzalez.nicolas@icloud.com)
✅ Imports et configuration initiale terminés.


In [29]:
# --- Fonctions Selenium et Extraction (Mises à jour) ---

def get_user_agent():
    """Retourne un User-Agent aléatoire"""
    user_agents = [
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36',
        'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.0 Safari/605.1.15',
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:123.0) Gecko/20100101 Firefox/123.0',
    ]
    return random.choice(user_agents)

def setup_driver():
    """Configure et retourne un driver Selenium Chrome"""
    print("\nConfiguration du navigateur Chrome (Selenium)...")
    chrome_options = Options()
    # Pour voir le navigateur, commentez la ligne suivante :
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument(f"user-agent={get_user_agent()}")
    chrome_options.add_argument("--disable-notifications")
    chrome_options.add_argument("--disable-extensions")
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--window-size=1920,1080")
    chrome_options.add_argument("--log-level=3")

    driver = None
    try:
        service = Service(ChromeDriverManager().install())
        driver = webdriver.Chrome(service=service, options=chrome_options)
        print("✅ Navigateur Chrome configuré.")
        return driver
    except ValueError as ve:
         print(f"❌ Erreur de configuration WebDriver: {ve}")
         return None
    except Exception as e:
        print(f"❌ Erreur inattendue config navigateur: {e}")
        if driver: driver.quit()
        return None

def linkedin_login(driver, username, password):
    """Se connecte à LinkedIn"""
    if not driver: return False
    print("\nTentative de connexion à LinkedIn...")
    try:
        login_url = "https://www.linkedin.com/login?fromSignIn=true&trk=guest_homepage-basic_nav-header-signin"
        driver.get(login_url)
        print(f" Accès à {login_url}")
        time.sleep(random.uniform(3, 5))

        if "feed" in driver.current_url:
            print(" Déjà connecté.")
            return True

        print(" Remplissage du formulaire...")
        try:
            username_field = WebDriverWait(driver, 15).until(EC.visibility_of_element_located((By.ID, "username")))
            password_field = driver.find_element(By.ID, "password")
            username_field.send_keys(username)
            time.sleep(random.uniform(0.5, 1.2))
            password_field.send_keys(password)
            time.sleep(random.uniform(0.5, 1.2))
            submit_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[data-litms-control-urn='login-submit'], button[type='submit']")))
            submit_button.click()
            print(" Bouton de connexion cliqué.")
            print(" Attente après soumission (5-8 sec)...")
            time.sleep(random.uniform(5, 8))
            current_url = driver.current_url
            if "feed" in current_url:
                print("✅ Connexion réussie.")
                return True
            elif "checkpoint" in current_url:
                print("⚠️ Connexion réussie mais checkpoint détecté.")
                return True # On essaie quand même
            else:
                print(f"❌ Échec de la connexion. URL: {current_url}")
                try:
                    error_element = driver.find_element(By.CSS_SELECTOR, "[role='alert'], .form__input--error, #error-for-password, #error-for-username")
                    if error_element: print(f" Message d'erreur: {error_element.text[:150]}")
                except NoSuchElementException: print(" Aucun message d'erreur trouvé.")
                return False
        except TimeoutException: print("❌ Timeout formulaire connexion."); return False
        except Exception as e: print(f"❌ Erreur formulaire : {e}"); return False
    except Exception as e: print(f"❌ Erreur majeure connexion : {e}"); return False

def get_page_content_with_selenium(url, driver):
    """Récupère le HTML de la page cible avec Selenium"""
    if not driver: return None
    print(f"\nAccès à l'URL cible via Selenium : {url}")
    try:
        driver.get(url)
        print(" Attente chargement (5-8 sec)...")
        time.sleep(random.uniform(5, 8))
        current_url = driver.current_url
        if "login" in current_url or "signup" in current_url or "authwall" in current_url:
             print(f"⚠️ Redirection vers {current_url}.")
             return None
        print(" Défilement léger...")
        for i in range(2):
             driver.execute_script(f"window.scrollTo(0, {(i + 1) * 600});")
             print(f" Scroll {i+1}/2...")
             time.sleep(random.uniform(1.5, 2.5))
        try:
             WebDriverWait(driver, 15).until(EC.visibility_of_element_located((By.TAG_NAME, "h1")))
             print(" Élément H1 visible.")
        except TimeoutException: print("⚠️ Timeout H1.")
        html_content = driver.page_source
        print("✅ Contenu HTML récupéré.")
        return html_content
    except Exception as e: print(f"❌ Erreur récupération page : {e}"); return None

# --- NOUVELLE VERSION ICI ---
def extract_article_content(html_content):
    """Extrait le contenu principal de l'article en utilisant le sélecteur .reader-article-content"""
    if not html_content: return "Contenu HTML non disponible"
    print("\nAnalyse HTML pour contenu article (avec sélecteur .reader-article-content)...")
    soup = BeautifulSoup(html_content, 'html.parser')
    article_text = ""
    found_by = "Aucune méthode"
    selector = 'div.reader-article-content' # <--- NOUVEAU SÉLECTEUR BASÉ SUR INSPECTION

    try:
        content_div = soup.select_one(selector)
        if content_div:
            content_parts = []
            # Itérer sur les éléments pour extraire le texte
            for element in content_div.find_all(['p', 'h1', 'h2', 'h3', 'h4', 'h5', 'h6', 'li'], recursive=True):
                 text_part = element.get_text(strip=True)
                 if text_part: content_parts.append(text_part)
            if content_parts:
                article_text = "\n\n".join(content_parts)
                found_by = f"Sélecteur '{selector}'"
                print(f" Contenu trouvé via: {found_by}")
                return article_text.strip()
            else: # Fallback texte brut
                 article_text = content_div.get_text(separator='\n', strip=True)
                 if article_text:
                      found_by = f"Sélecteur '{selector}' (texte brut)"
                      print(f" Contenu trouvé via: {found_by}")
                      return article_text.strip()
    except Exception as e: print(f" Erreur en cherchant contenu avec '{selector}': {e}"); pass

    if not article_text:
        print("⚠️ Contenu article non trouvé avec nouveau sélecteur.")
        return "Contenu non trouvé"
    return "Contenu non trouvé (fin)" # Ne devrait pas arriver
# --- FIN NOUVELLE VERSION ---

def extract_likes_count_with_selenium(driver):
    """Extrait le nombre de likes/réactions avec Selenium (méthode aria-label)"""
    print("\nRecherche des Likes/Réactions...")
    try:
        xpath_selector = "//button[contains(@aria-label, 'réaction') or contains(@aria-label, 'reaction')]"
        likes_element = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, xpath_selector)))
        aria_label_text = likes_element.get_attribute("aria-label")
        match = re.search(r'^(\d+)', aria_label_text)
        if match:
            likes_count = match.group(1)
            print(f" Likes trouvés via XPath aria-label: '{xpath_selector}' -> {likes_count}")
            return likes_count
        else:
             likes_text = likes_element.get_attribute("textContent").strip()
             match_text = re.search(r'^(\d+)', likes_text)
             if match_text:
                  likes_count = match_text.group(1); print(f" Likes trouvés via XPath (texte fallback): '{xpath_selector}' -> {likes_count}"); return likes_count
    except (NoSuchElementException, TimeoutException): print("⚠️ Élément Likes non trouvé via XPath.")
    except Exception as e: print(f" Erreur mineure likes XPath: {e}")
    # Fallback CSS (au cas où)
    try:
        selector_css = ".social-details-social-counts__reactions-count"; likes_element_css = WebDriverWait(driver, 5).until(EC.visibility_of_element_located((By.CSS_SELECTOR, selector_css))); likes_text_css = likes_element_css.get_attribute("textContent").strip(); match_css = re.search(r'^(\d+)', likes_text_css)
        if match_css: likes_count_css = match_css.group(1); print(f" Likes trouvés via CSS fallback: '{selector_css}' -> {likes_count_css}"); return likes_count_css
    except Exception: print("⚠️ Likes non trouvés via CSS non plus.")
    return "Non trouvé"

def extract_comments_count_with_selenium(driver):
    """Extrait le nombre de commentaires avec Selenium (méthode aria-label)"""
    print("\nRecherche des Commentaires...")
    try:
        xpath_selector = "//button[contains(@aria-label, 'commentaire') or contains(@aria-label, 'comment')]"
        # Alternative : xpath_selector = "//a[contains(@aria-label, 'commentaire') or contains(@aria-label, 'comment')]"
        comments_element = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, xpath_selector)))
        aria_label_text = comments_element.get_attribute("aria-label")
        match = re.search(r'^(\d+)', aria_label_text)
        if match:
            comments_count = match.group(1); print(f" Commentaires trouvés via XPath aria-label: '{xpath_selector}' -> {comments_count}"); return comments_count
        else:
             comments_text = comments_element.get_attribute("textContent").strip(); match_text = re.search(r'^(\d+)', comments_text)
             if match_text: comments_count = match_text.group(1); print(f" Commentaires trouvés via XPath (texte fallback): '{xpath_selector}' -> {comments_count}"); return comments_count
    except (NoSuchElementException, TimeoutException): print("⚠️ Élément Commentaires non trouvé via XPath.")
    except Exception as e: print(f" Erreur mineure commentaires XPath: {e}")
    # Ajouter fallbacks CSS si nécessaire
    print("⚠️ Nombre de commentaires non trouvé via sélecteurs principaux.")
    return "Non trouvé"

print("✅ Fonctions Selenium (avec extract_article_content mise à jour) définies.")

✅ Fonctions Selenium (avec extract_article_content mise à jour) définies.


cellule 3

In [31]:
# --- Logique Principale d'Exécution ---

# Imports nécessaires au cas où la cellule est exécutée seule (normalement faits en Cellule 1)
import os
import re
import time
# Assurez-vous que les variables LINKEDIN_USERNAME et LINKEDIN_PASSWORD sont chargées (normalement fait en Cellule 1)
# Assurez-vous que les fonctions (setup_driver, linkedin_login, get_page_content_with_selenium, etc.) sont définies (normalement fait en Cellule 2)

urls_filename = "linkedin_urls_found.txt" # Fichier créé par Phase 1
target_url = None
driver = None # Important d'initialiser à None

# --- Charger l'URL cible (la deuxième) ---
linkedin_urls = []
# Vérifier si le fichier existe dans le dossier courant (normalement 01_collecte_donnees)
if os.path.exists(urls_filename):
    try:
        with open(urls_filename, "r") as f:
            linkedin_urls = [line.strip() for line in f if line.strip()]
        if len(linkedin_urls) >= 2:
            target_url = linkedin_urls[1] # CIBLER LA DEUXIÈME URL
            print(f"URL Cible pour ce test : {target_url}")
        elif linkedin_urls:
             print(f"⚠️ Fichier '{urls_filename}' contient seulement {len(linkedin_urls)} URL(s). Impossible de tester la deuxième.")
             target_url = None
        else:
            print(f"⚠️ Le fichier '{urls_filename}' est vide.")
            target_url = None
    except Exception as e:
        print(f"❌ Erreur lors de la lecture du fichier URLs : {e}")
        target_url = None
else:
    print(f"❌ Fichier URLs '{urls_filename}' non trouvé.")
    target_url = None

# --- Exécuter seulement si on a une URL cible et les identifiants ---
# Vérifier aussi que les fonctions existent (au cas où Cellule 2 n'a pas été exécutée)
if target_url and 'LINKEDIN_USERNAME' in locals() and LINKEDIN_USERNAME and 'LINKEDIN_PASSWORD' in locals() and LINKEDIN_PASSWORD and 'setup_driver' in globals():
    article_content = "Non extrait"
    likes_count = "Non extrait"
    comments_count = "Non extrait"
    resultat_url = {} # Initialiser le dictionnaire de résultat

    try:
        # 1. Démarrer le navigateur
        driver = setup_driver() # Défini dans la cellule précédente

        if driver:
            # 2. Se connecter
            login_success = linkedin_login(driver, LINKEDIN_USERNAME, LINKEDIN_PASSWORD) # Défini dans la cellule précédente

            if login_success:
                # 3. Récupérer le HTML de la page cible
                html_content = get_page_content_with_selenium(target_url, driver) # Défini dans la cellule précédente

                if html_content:
                    # 4. Extraire le contenu texte
                    # Utilise la fonction MISE À JOUR définie précédemment
                    article_content = extract_article_content(html_content)
                    # 5. Extraire Likes et Commentaires (via Selenium sur la page chargée)
                    likes_count = extract_likes_count_with_selenium(driver) # Défini dans la cellule précédente
                    comments_count = extract_comments_count_with_selenium(driver) # Défini dans la cellule précédente
                else:
                    article_content = "Erreur: Impossible de récupérer le HTML de la page cible."
                    # Assigner aussi les erreurs aux autres comptes si HTML échoue
                    likes_count = "Erreur: HTML non récupéré"
                    comments_count = "Erreur: HTML non récupéré"
            else:
                article_content = "Erreur: Échec de la connexion LinkedIn."
                likes_count = "Erreur: Connexion échouée"
                comments_count = "Erreur: Connexion échouée"
        else:
             article_content = "Erreur: Driver Selenium non initialisé."
             likes_count = "Erreur: Driver non initialisé"
             comments_count = "Erreur: Driver non initialisé"

    except Exception as e:
        print(f"❌ Une erreur majeure est survenue lors de l'exécution principale : {e}")
        article_content = f"Erreur: {e}"
        # Assurer que les autres variables ont aussi une valeur d'erreur
        if 'likes_count' not in locals() or likes_count == "Non extrait": likes_count = f"Erreur: {e}"
        if 'comments_count' not in locals() or comments_count == "Non extrait": comments_count = f"Erreur: {e}"
    finally:
        # 6. Fermer le navigateur proprement, quoi qu'il arrive
        if driver:
            try:
                driver.quit()
                print("\n✅ Navigateur Selenium fermé.")
            except Exception as e:
                print(f"⚠️ Erreur lors de la fermeture du navigateur: {e}")

    # --- Stocker et Afficher les résultats finaux pour cette URL ---
    # Utilisation de la logique modifiée pour créer et afficher le dictionnaire
    resultat_url = {
        "url": target_url if target_url else "URL non définie",
        "likes": likes_count, # Contient déjà la valeur ou un message d'erreur
        "commentaires": comments_count, # Contient déjà la valeur ou un message d'erreur
        "contenu": article_content if isinstance(article_content, str) else f"Erreur: {article_content}" # Stocker l'erreur si contenu non str
    }

    print("\n" + "="*50)
    print("RÉSULTAT STRUCTURÉ POUR L'URL TESTÉE (Dictionnaire)")
    print("="*50)
    # Afficher les clés et valeurs du dictionnaire pour vérification
    print(f"- URL: {resultat_url.get('url', 'N/A')}")
    print(f"- Likes: {resultat_url.get('likes', 'N/A')}")
    print(f"- Commentaires: {resultat_url.get('commentaires', 'N/A')}")
    contenu_extrait = resultat_url.get('contenu', '')
    # Gérer le cas où le contenu est une erreur et n'a pas de len()
    if isinstance(contenu_extrait, str):
        print(f"- Contenu (Extrait): {contenu_extrait[:200] + '...' if len(contenu_extrait) > 200 else contenu_extrait}")
    else:
        print(f"- Contenu (Erreur): {contenu_extrait}")
    print("="*50)

    # Note : Dans l'étape suivante (boucle), on ajoutera ce dictionnaire 'resultat_url'
    # à une liste globale : all_results.append(resultat_url)

# Gérer les cas où l'exécution n'a pas eu lieu car target_url était None ou identifiants manquants
elif not target_url:
    print("\nAucune URL cible à traiter (vérifiez le fichier d'URLs ou le code de chargement).")
elif not ('LINKEDIN_USERNAME' in locals() and LINKEDIN_USERNAME and 'LINKEDIN_PASSWORD' in locals() and LINKEDIN_PASSWORD):
     print("\nIdentifiants LinkedIn manquants, impossible de continuer (vérifiez l'exécution de la Cellule 1 et le fichier .env).")
else:
     print("\nProblème inconnu avant le démarrage de Selenium.")

URL Cible pour ce test : https://fr.linkedin.com/pulse/derni%C3%A8res-avanc%C3%A9es-en-ia-g%C3%A9n%C3%A9rative-mars-2025-arnault-chatel-zs39e

Configuration du navigateur Chrome (Selenium)...
✅ Navigateur Chrome configuré.

Tentative de connexion à LinkedIn...
 Accès à https://www.linkedin.com/login?fromSignIn=true&trk=guest_homepage-basic_nav-header-signin
 Remplissage du formulaire...
 Bouton de connexion cliqué.
 Attente après soumission (5-8 sec)...
✅ Connexion réussie.

Accès à l'URL cible via Selenium : https://fr.linkedin.com/pulse/derni%C3%A8res-avanc%C3%A9es-en-ia-g%C3%A9n%C3%A9rative-mars-2025-arnault-chatel-zs39e
 Attente chargement (5-8 sec)...
 Défilement léger...
 Scroll 1/2...
 Scroll 2/2...
 Élément H1 visible.
✅ Contenu HTML récupéré.

Analyse HTML pour contenu article (avec sélecteur .reader-article-content)...
 Contenu trouvé via: Sélecteur 'div.reader-article-content'

Recherche des Likes/Réactions...
 Likes trouvés via XPath aria-label: '//button[contains(@aria-lab

maintnenat on lance le grand test 

In [32]:
# --- Logique Principale d'Exécution (Boucle sur les URLs et Sauvegarde CSV) ---

# Imports supplémentaires nécessaires
import csv
import os
import re
import time
import random
# Assurez-vous que les variables LINKEDIN_USERNAME/PASSWORD sont chargées (Cellule 1)
# Assurez-vous que les fonctions Selenium/Extraction sont définies (Cellule 2)

# --- Configuration ---
urls_filename = "linkedin_urls_found.txt"
output_csv_filename = "linkedin_extracted_data.csv" # Nom du fichier de sortie CSV
urls_to_process = [] # Liste des URLs à traiter
all_results = [] # Liste pour stocker tous les dictionnaires de résultats
driver = None # Initialiser

# --- Charger TOUTES les URLs ---
# Vérifier si le fichier existe dans le dossier courant (normalement 01_collecte_donnees)
if os.path.exists(urls_filename):
    try:
        with open(urls_filename, "r") as f:
            # Lire seulement les lignes valides contenant 'linkedin.com'
            urls_to_process = [line.strip() for line in f if line.strip() and 'linkedin.com' in line]
        if urls_to_process:
            print(f"✅ {len(urls_to_process)} URLs chargées depuis '{urls_filename}' pour traitement.")
            # Optionnel: Limiter le nombre pour tester la boucle rapidement
            # urls_to_process = urls_to_process[:3] # Prend seulement les 3 premières
            # print(f" -> Limitation à {len(urls_to_process)} URLs pour ce test.")
        else:
            print(f"⚠️ Le fichier '{urls_filename}' est vide ou ne contient pas d'URLs valides.")
            urls_to_process = [] # Assurer liste vide
    except Exception as e:
        print(f"❌ Erreur lors de la lecture du fichier URLs : {e}")
        urls_to_process = []
else:
    print(f"❌ Fichier URLs '{urls_filename}' non trouvé.")
    urls_to_process = []

# --- Exécuter seulement si on a des URLs et les identifiants ---
if urls_to_process and 'LINKEDIN_USERNAME' in locals() and LINKEDIN_USERNAME and 'LINKEDIN_PASSWORD' in locals() and LINKEDIN_PASSWORD and 'setup_driver' in globals():

    try:
        # 1. Démarrer le navigateur UNE SEULE FOIS
        print("\n--- Démarrage du processus de scraping en boucle ---")
        driver = setup_driver() # Fonction définie en Cellule 2

        if driver:
            # 2. Se connecter UNE SEULE FOIS
            login_success = linkedin_login(driver, LINKEDIN_USERNAME, LINKEDIN_PASSWORD) # Fonction définie en Cellule 2

            if login_success:
                # 3. Boucler sur chaque URL
                for index, target_url in enumerate(urls_to_process):
                    print("\n" + "-"*30)
                    print(f"Traitement URL {index + 1}/{len(urls_to_process)} : {target_url}")
                    print("-"*30)

                    # Initialiser les résultats pour cette URL
                    article_content = "Non extrait"
                    likes_count = "Non extrait"
                    comments_count = "Non extrait"
                    # Pas de date ici

                    try:
                        # Récupérer le HTML de la page cible
                        html_content = get_page_content_with_selenium(target_url, driver) # Fonction définie en Cellule 2

                        if html_content:
                            # Extraire les informations
                            article_content = extract_article_content(html_content) # Fonction définie en Cellule 2
                            likes_count = extract_likes_count_with_selenium(driver) # Fonction définie en Cellule 2
                            comments_count = extract_comments_count_with_selenium(driver) # Fonction définie en Cellule 2
                        else:
                            # Marquer toutes les infos comme erronées si HTML non récupéré
                            error_msg = "Erreur: HTML non récupéré"
                            article_content, likes_count, comments_count = error_msg, error_msg, error_msg

                        # Stocker le résultat de cette URL dans un dictionnaire
                        resultat_url = {
                            "url": target_url,
                            # "date_publication": publication_date, # Pas de date
                            "likes": likes_count,
                            "commentaires": comments_count,
                            "contenu": article_content if isinstance(article_content, str) else f"Erreur: {article_content}"
                        }
                        # Ajouter ce dictionnaire à la liste globale
                        all_results.append(resultat_url)
                        print(f"-> Résultat ajouté pour {target_url}")
                        print(f"   Likes: {likes_count}, Commentaires: {comments_count}")

                    except Exception as e_url:
                        # Gérer une erreur spécifique à cette URL sans arrêter la boucle
                        print(f"❌ Erreur majeure lors du traitement de l'URL {target_url}: {e_url}")
                        resultat_url = {"url": target_url, "likes": "Erreur", "commentaires": "Erreur", "contenu": f"Erreur scraping: {e_url}"}
                        all_results.append(resultat_url)

                    # Pause INDISPENSABLE entre chaque URL
                    sleep_time = random.uniform(10, 20) # Pause de 10 à 20 secondes (ajustable)
                    print(f"--- Pause de {sleep_time:.1f} secondes avant la prochaine URL ---")
                    time.sleep(sleep_time)

            else: # Si la connexion initiale a échoué
                print("❌ Échec de la connexion LinkedIn initiale, arrêt du traitement des URLs.")

    except Exception as e_main:
        print(f"❌ Une erreur majeure est survenue lors de l'exécution principale de la boucle : {e_main}")
    finally:
        # Fermer le navigateur UNE SEULE FOIS à la fin
        if driver:
            try:
                driver.quit()
                print("\n✅ Navigateur Selenium fermé après la boucle.")
            except Exception as e:
                print(f"⚠️ Erreur lors de la fermeture du navigateur: {e}")

    # --- 4. Sauvegarder tous les résultats collectés en CSV ---
    print("\n" + "="*50)
    print(f"FIN DU SCRAPING : {len(all_results)} URLs traitées.")
    print("="*50)

    if all_results:
        print(f"Tentative de sauvegarde des résultats dans {output_csv_filename}...")
        try:
            # Définir les noms des colonnes (l'ordre est important)
            fieldnames = ['url', 'likes', 'commentaires', 'contenu'] # Sans la date
            with open(output_csv_filename, 'w', newline='', encoding='utf-8') as csvfile:
                writer = csv.DictWriter(csvfile, fieldnames=fieldnames, extrasaction='ignore')
                writer.writeheader() # Écrire l'en-tête
                writer.writerows(all_results) # Écrire les données
            print(f"✅ Résultats sauvegardés avec succès dans '{output_csv_filename}' !")
        except Exception as e_save:
            print(f"❌ Erreur lors de la sauvegarde du fichier CSV : {e_save}")
    else:
        print("Aucun résultat n'a été collecté pour la sauvegarde.")

# Gérer les cas où on ne peut pas démarrer
elif not urls_to_process:
    print("\nAucune URL à traiter trouvée dans le fichier.")
elif not ('LINKEDIN_USERNAME' in locals() and LINKEDIN_USERNAME and 'LINKEDIN_PASSWORD' in locals() and LINKEDIN_PASSWORD):
     print("\nIdentifiants LinkedIn manquants.")
else:
     print("\nProblème inconnu avant démarrage.")

✅ 10 URLs chargées depuis 'linkedin_urls_found.txt' pour traitement.

--- Démarrage du processus de scraping en boucle ---

Configuration du navigateur Chrome (Selenium)...
✅ Navigateur Chrome configuré.

Tentative de connexion à LinkedIn...
 Accès à https://www.linkedin.com/login?fromSignIn=true&trk=guest_homepage-basic_nav-header-signin
 Remplissage du formulaire...
 Bouton de connexion cliqué.
 Attente après soumission (5-8 sec)...
✅ Connexion réussie.

------------------------------
Traitement URL 1/10 : https://fr.linkedin.com/pulse/comment-arbitrer-entre-ia-classique-et-g%C3%A9n%C3%A9rative-le-de-mascureau-2drce
------------------------------

Accès à l'URL cible via Selenium : https://fr.linkedin.com/pulse/comment-arbitrer-entre-ia-classique-et-g%C3%A9n%C3%A9rative-le-de-mascureau-2drce
 Attente chargement (5-8 sec)...
 Défilement léger...
 Scroll 1/2...
 Scroll 2/2...
 Élément H1 visible.
✅ Contenu HTML récupéré.

Analyse HTML pour contenu article (avec sélecteur .reader-articl

phase 3

In [33]:
# --- Phase 3 : Filtrage des Données et Formatage pour Fine-Tuning ---

import pandas as pd
import json
import os
import numpy as np # numpy est souvent une dépendance de pandas

# --- Configuration ---
input_csv_filename = "linkedin_extracted_data.csv" # Fichier CSV créé par Phase 2
output_jsonl_filename = "finetuning_data_final.jsonl" # Fichier final pour OpenAI

# --- 3a: Charger et Préparer les Données ---
print(f"Chargement des données depuis {input_csv_filename}...")
# Vérifier l'existence du fichier dans le dossier courant (normalement 01_collecte_donnees)
if not os.path.exists(input_csv_filename):
    print(f"❌ ERREUR: Le fichier {input_csv_filename} n'existe pas. Exécutez d'abord le script de scraping (Phase 2 - Cellule 3 avec boucle).")
    df = pd.DataFrame() # Créer DataFrame vide pour éviter erreurs plus loin
else:
    try:
        df = pd.read_csv(input_csv_filename)
        print(f"✅ {len(df)} lignes chargées.")

        # Afficher les premières lignes et les types de données
        # print("\nAperçu des données brutes (5 premières lignes) :")
        # print(df.head())
        # print("\nTypes de données initiaux :")
        # print(df.info()) # Décommentez pour plus de détails

        # --- Nettoyage Likes/Commentaires ---
        print("\nNettoyage des colonnes Likes/Commentaires...")
        # Convertir en numérique, mettre NaN si erreur
        # Utiliser errors='coerce' transforme les erreurs en NaN (Not a Number)
        df['likes_num'] = pd.to_numeric(df['likes'], errors='coerce')
        df['commentaires_num'] = pd.to_numeric(df['commentaires'], errors='coerce')

        # Optionnel: Afficher combien de lignes avaient des erreurs de conversion
        likes_errors = df['likes_num'].isna().sum() - df['likes'].isna().sum() # Compte les nouvelles NaN créées par coerce
        comments_errors = df['commentaires_num'].isna().sum() - df['commentaires'].isna().sum()
        if likes_errors > 0: print(f"   {likes_errors} valeur(s) 'likes' non numériques trouvées (seront mises à 0).")
        if comments_errors > 0: print(f"   {comments_errors} valeur(s) 'commentaires' non numériques trouvées (seront mises à 0).")

        # Remplacer les NaN (erreurs de conversion ou valeurs manquantes initiales) par 0 et convertir en entier
        df['likes_clean'] = df['likes_num'].fillna(0).astype(int)
        df['commentaires_clean'] = df['commentaires_num'].fillna(0).astype(int)

        # --- Nettoyage Contenu ---
        print("\nNettoyage de la colonne Contenu...")
        # Enlever les lignes où le contenu est manquant ou indique une erreur claire
        initial_rows = len(df)
        # S'assurer que 'contenu' est de type string avant d'utiliser .str
        df['contenu'] = df['contenu'].astype(str)
        # Exclure les lignes contenant les marqueurs d'erreur ou de contenu non trouvé
        error_patterns = "Erreur|Non extrait|Contenu non trouvé|HTML non récupéré"
        df = df[df['contenu'].notna() & (~df['contenu'].str.contains(error_patterns, na=False, case=False, regex=True))]
        rows_after_content_clean = len(df)
        print(f"   {initial_rows - rows_after_content_clean} lignes supprimées à cause de contenu invalide/erreur.")
        print(f"   {rows_after_content_clean} lignes restantes avec contenu potentiellement valide.")

        # Afficher un résumé statistique des colonnes numériques propres des lignes restantes
        if not df.empty:
             print("\nStatistiques des Likes/Commentaires (sur données nettoyées) :")
             # Afficher les stats uniquement pour les lignes où le contenu est valide
             print(df[['likes_clean', 'commentaires_clean']].describe())
        else:
             print("\nDataFrame vide après nettoyage du contenu.")

    except Exception as e:
        print(f"❌ Erreur lors du chargement ou nettoyage des données CSV : {e}")
        df = pd.DataFrame() # Assurer que df est vide si erreur

# --- 3b & 3c: Définir Critères et Filtrer ---
# Continuer seulement si le DataFrame df n'est pas vide après chargement/nettoyage
if 'df' in locals() and not df.empty:
    # --- !!! À ADAPTER PAR VOUS !!! ---
    # Définissez ici vos critères pour un post "pertinent" / de "qualité"
    # REGARDEZ LES STATS CI-DESSUS pour fixer des seuils réalistes pour VOTRE échantillon
    # Mettez des valeurs basses pour être sûr d'avoir des résultats avec seulement 10 URLs initiales
    SEUIL_MIN_LIKES = 10       # EXEMPLE : au moins 10 likes (à adapter !)
    SEUIL_MIN_COMMENTAIRES = 2 # EXEMPLE : au moins 2 commentaires (à adapter !)
    # ---------------------------------

    print(f"\nFiltrage des posts : Likes >= {SEUIL_MIN_LIKES} ET Commentaires >= {SEUIL_MIN_COMMENTAIRES}...")

    df_filtered = df[
        (df['likes_clean'] >= SEUIL_MIN_LIKES) &
        (df['commentaires_clean'] >= SEUIL_MIN_COMMENTAIRES)
    ].copy() # .copy() pour éviter les warnings

    print(f"✅ {len(df_filtered)} posts correspondent aux critères de filtrage.")

    if not df_filtered.empty:
        # print("Aperçu des posts filtrés (index, likes, commentaires) :")
        # print(df_filtered[['likes_clean', 'commentaires_clean']].head()) # Décommentez pour voir

        # --- 3d: Formater en JSONL pour OpenAI ---
        print(f"\nFormatage des {len(df_filtered)} posts filtrés en JSONL pour '{output_jsonl_filename}'...")

        # Adaptez ce prompt système si vous voulez que le modèle apprenne un style particulier
        system_prompt = "Vous êtes un expert en IA rédigeant des posts pour LinkedIn dans un style professionnel, engageant et informatif."

        lines_written = 0
        try:
            with open(output_jsonl_filename, 'w', encoding='utf-8') as f_out:
                # Itérer sur le DataFrame filtré
                for index, row in df_filtered.iterrows():
                    post_content = str(row['contenu']) # Assurer que c'est une string

                    # Ajouter une vérification de longueur minimale si souhaité
                    if len(post_content) > 150: # EXEMPLE: ignorer posts très courts (parfois juste un titre)
                        message = { "messages": [
                                {"role": "system", "content": system_prompt},
                                {"role": "user", "content": ""}, # Prompt utilisateur vide si on fine-tune le style/sujet
                                {"role": "assistant", "content": post_content} # Le post réel est la réponse désirée
                            ] }
                        # Écrire chaque message comme une ligne JSON
                        f_out.write(json.dumps(message, ensure_ascii=False) + "\n")
                        lines_written += 1
                    else:
                        # Optionnel : Informer si un post est ignoré car trop court
                        # print(f"   Info: Post à l'index {index} ignoré car trop court (longueur {len(post_content)}).")
                        pass # Ne rien faire pour les posts trop courts


            print(f"✅ {lines_written} posts formatés et sauvegardés dans '{output_jsonl_filename}'.")
            if lines_written > 0:
                 print("   Ce fichier est prêt pour le fine-tuning OpenAI !")
            elif len(df_filtered) > 0: # Si des posts ont été filtrés mais aucun écrit
                 print("   Aucun post n'a été écrit (vérifiez le seuil de longueur minimale ou le contenu).")

        except Exception as e:
            print(f"❌ Erreur lors du formatage ou de la sauvegarde JSONL : {e}")
    # Fin de "if not df_filtered.empty:"
    elif len(df) > 0: # Si le df initial n'était pas vide mais qu'aucun post n'a passé le filtre
         print("\nAucun post ne correspond aux critères de filtrage définis.")
# Fin de "if 'df' in locals() and not df.empty:"
else:
    print("\nLe DataFrame est vide ou n'a pas pu être chargé/nettoyé, impossible de continuer la Phase 3.")

Chargement des données depuis linkedin_extracted_data.csv...
✅ 10 lignes chargées.

Nettoyage des colonnes Likes/Commentaires...
   2 valeur(s) 'commentaires' non numériques trouvées (seront mises à 0).

Nettoyage de la colonne Contenu...
   3 lignes supprimées à cause de contenu invalide/erreur.
   7 lignes restantes avec contenu potentiellement valide.

Statistiques des Likes/Commentaires (sur données nettoyées) :
       likes_clean  commentaires_clean
count     7.000000            7.000000
mean     20.285714            1.714286
std      26.010987            1.496026
min       3.000000            0.000000
25%       8.000000            0.500000
50%      10.000000            2.000000
75%      17.500000            2.500000
max      78.000000            4.000000

Filtrage des posts : Likes >= 10 ET Commentaires >= 2...
✅ 1 posts correspondent aux critères de filtrage.

Formatage des 1 posts filtrés en JSONL pour 'finetuning_data_final.jsonl'...
✅ 1 posts formatés et sauvegardés dans 'fin

### Phase 3 : Filtrage des Données et Formatage - Résultats

**Objectif :** Traiter le fichier CSV (`linkedin_extracted_data.csv`) contenant les données brutes scrapées en Phase 2, afin de sélectionner les posts les plus pertinents (basés sur l'engagement) et de les formater pour le fine-tuning OpenAI.

**Actions Réalisées par le Script :**

1.  **Chargement :** 10 lignes ont été chargées depuis `linkedin_extracted_data.csv`.
2.  **Nettoyage Likes/Commentaires :** Les colonnes ont été converties en nombres (2 erreurs de conversion pour les commentaires ont été gérées et mises à 0).
3.  **Nettoyage Contenu :** Les lignes contenant des erreurs de scraping ou un contenu invalide ont été supprimées (3 lignes enlevées). Il restait 7 posts avec du contenu potentiellement valide.
4.  **Statistiques :** Des statistiques descriptives ont été calculées sur les 7 posts valides (Likes : max 75, moyenne ~20 ; Commentaires : max 4, moyenne ~1.7).
5.  **Filtrage :** Un filtre a été appliqué pour ne garder que les posts avec **Likes >= 10 ET Commentaires >= 2** (basé sur les seuils définis dans le code).
6.  **Résultat du Filtrage :** **1 post** correspondait à ces critères sur l'échantillon de 7 posts valides.
7.  **Formatage :** Ce post unique a été formaté en JSONL selon la structure attendue par l'API OpenAI (messages système/utilisateur/assistant).
8.  **Sauvegarde :** Le résultat formaté a été sauvegardé dans le fichier `finetuning_data_final.jsonl`.

**Conclusion de la Phase 3 :**

* Le pipeline complet (Chargement -> Nettoyage -> Filtrage -> Formatage) fonctionne correctement !
* Le fichier `finetuning_data_final.jsonl` a été créé et est techniquement prêt à être utilisé pour un fine-tuning sur OpenAI.

---

**Prochaine Étape Stratégique : Passage à l'Échelle**

Le processus est validé, mais le jeu de données final ne contient qu'un seul exemple, ce qui est insuffisant pour un fine-tuning efficace. La prochaine étape essentielle est de **retourner à la Phase 1 pour collecter une quantité beaucoup plus importante d'URLs pertinentes**, afin d'alimenter ce pipeline et générer un fichier `finetuning_data_final.jsonl` contenant plusieurs centaines (ou milliers) d'exemples de haute qualité.

on essaie maintnenat de recuperer plus de données

In [34]:
# --- Phase 1 : Découverte d'URLs via Google Search (Version Multi-Requêtes) ---

# Imports nécessaires
from googlesearch import search
import time
import os
import random

# --- Configuration ---

# !!! MODIFIEZ ET COMPLÉTEZ CETTE LISTE SELON VOS BESOINS !!!
# Ajoutez des mots-clés précis, des sujets spécifiques, des noms d'experts...
search_queries = [
    'site:linkedin.com/pulse/ ("intelligence artificielle" OR "IA générative")',
    'site:linkedin.com/pulse/ ("machine learning" OR "deep learning" applications)',
    'site:linkedin.com/pulse/ ("EU AI Act" OR "Acte IA Europe" OR "régulation IA")',
    'site:linkedin.com/pulse/ ("prompt engineering" OR "ingénierie de prompt" OR "chatbot")',
    'site:linkedin.com/pulse/ (IA OR "intelligence artificielle" entreprise OR PME)',
    'site:linkedin.com/pulse/ (éthique IA OR "AI ethics" OR "biais algorithmique")',
    'site:linkedin.com/pulse/ (MLOps OR "mise en production IA")',
    # Ajoutez d'autres requêtes ici ! Exemple :
    # 'site:linkedin.com/pulse/ ("traitement langage naturel" OR NLP)',
    # 'site:linkedin.com/posts/ ("intelligence artificielle" OR "IA générative" OR "machine learning")', # Posts standards (moins fiable)
]

# Limite MANUELLE du nombre de résultats Google à traiter PAR REQUÊTE
# Mettez une valeur plus haute pour tenter d'avoir plus de résultats par recherche,
# même si la bibliothèque simple peut être limitée. Essayons 30.
results_limit_per_query = 30
# Délai entre les requêtes à Google (IMPORTANT)
pause_duration = 5.0 # Garder une pause raisonnable (5 sec ou plus)

output_filename = "linkedin_urls_found.txt" # Ce fichier sera ÉCRASÉ

# --- Stockage des URLs ---
all_found_urls = set() # Utilise un set pour éviter les doublons globaux

print(f"--- Démarrage de la recherche Google Multi-Requêtes ---")
print(f"Nombre de requêtes à exécuter : {len(search_queries)}")
print(f"Limite de traitement par requête : {results_limit_per_query}")
print(f"Pause entre requêtes Google : {pause_duration} secondes")
print(f"Fichier de sortie : {output_filename}")

# --- Boucle sur chaque requête ---
for i, query in enumerate(search_queries):
    print("\n" + "="*30)
    print(f"Requête {i+1}/{len(search_queries)} : '{query}'")
    print("="*30)

    processed_for_this_query = 0
    try:
        # Appel simplifié
        for url in search(query, pause=pause_duration, num_results=results_limit_per_query): # Ajout de num_results ici
            processed_for_this_query += 1
            # Filtrer pour ne garder que les URLs LinkedIn pertinentes
            if 'linkedin.com/' in url:
                # Exclure les pages non désirées
                excluded_paths = ['/groups/', '/jobs/', '/company/', '/school/', '/showcase/', '/talent/', '/legal/', '/directory/', '/feed/']
                if not any(excluded in url for excluded in excluded_paths):
                    if url not in all_found_urls:
                      print(f"  -> Trouvé ({processed_for_this_query}): {url}")
                      all_found_urls.add(url)

            # Arrêter si on a traité assez de résultats pour CETTE requête
            # Note: la bibliothèque peut s'arrêter avant si elle ne trouve plus rien
            if processed_for_this_query >= results_limit_per_query:
                print(f" Limite de {results_limit_per_query} résultats Google traités pour cette requête.")
                break
        # Petite pause supplémentaire entre les grosses requêtes
        print(f" Fin de la requête {i+1}. Pause avant la suivante...")
        time.sleep(random.uniform(5, 10)) # Pause plus longue entre les requêtes

    except Exception as e:
        print(f"\n❌ Une erreur est survenue pendant la recherche pour la requête {i+1} : {e}")
        print("   Cela peut être dû à un blocage temporaire de Google. Passage à la requête suivante...")
        time.sleep(random.uniform(10, 20)) # Pause plus longue en cas d'erreur
        continue # Passer à la requête suivante

# --- Afficher et Sauvegarder les résultats ---
print("\n" + "="*50)
print(f"RECHERCHE GOOGLE TERMINÉE")
print(f"--- Total URLs LinkedIn Uniques Trouvées (toutes requêtes confondues): {len(all_found_urls)} ---")
print("="*50)

if all_found_urls:
    print(f"\nTentative de sauvegarde des {len(all_found_urls)} URLs dans {output_filename}...")
    try:
        # Écraser l'ancien fichier avec les nouveaux résultats
        with open(output_filename, 'w', encoding='utf-8') as f:
            for url in sorted(list(all_found_urls)):
                f.write(url + "\n")
        print(f"✅ URLs sauvegardées avec succès dans '{output_filename}' !")
    except Exception as e_save:
        print(f"❌ Erreur lors de la sauvegarde du fichier URLs : {e_save}")
else:
    print("\nAucune URL LinkedIn pertinente n'a été trouvée au total.")

print("\n--- Prochaine Étape : Relancer les cellules de Phase 2 (Scraping Selenium) et Phase 3 (Filtrage/Formatage) avec ce nouveau fichier d'URLs ---")

--- Démarrage de la recherche Google Multi-Requêtes ---
Nombre de requêtes à exécuter : 7
Limite de traitement par requête : 30
Pause entre requêtes Google : 5.0 secondes
Fichier de sortie : linkedin_urls_found.txt

Requête 1/7 : 'site:linkedin.com/pulse/ ("intelligence artificielle" OR "IA générative")'

❌ Une erreur est survenue pendant la recherche pour la requête 1 : search() got an unexpected keyword argument 'pause'
   Cela peut être dû à un blocage temporaire de Google. Passage à la requête suivante...

Requête 2/7 : 'site:linkedin.com/pulse/ ("machine learning" OR "deep learning" applications)'

❌ Une erreur est survenue pendant la recherche pour la requête 2 : search() got an unexpected keyword argument 'pause'
   Cela peut être dû à un blocage temporaire de Google. Passage à la requête suivante...

Requête 3/7 : 'site:linkedin.com/pulse/ ("EU AI Act" OR "Acte IA Europe" OR "régulation IA")'

❌ Une erreur est survenue pendant la recherche pour la requête 3 : search() got an un

le defi majeur a ete de trouver une solution pour scrapper un volume important d'urls

je decide de tester via scrapping dog 

malheureusement echec 

etape 2